# Problem 1

Notation:

* $\tau \equiv T-t$: time to maturity (in years)
* $S(t)$: spot value for price
* $\sigma$: standard deviation of log-returns assuming Black-Scholes model
* $\Delta t = \tau/n$: discretized time step
* $u = \exp(\sigma \sqrt{\Delta T})$: increase factor for spot; $d = 1/u$ decrease factor

## a) Build pricer

Settings:

In [66]:
n_steps = 10
tau = 1.0 
dt = tau/n_steps
sigma = 0.30

spot = 1
risk_free_rate = 0.03

assert 0<=p <= 1

**First step**: build tree

In [67]:
def build_stock_tree(spot, tau, n_steps):
    
    # parameters
    dt = tau/n_steps
    u = np.exp(sigma*np.sqrt(dt))
    d = 1.0/u
    p = (np.exp(risk_free_rate*dt)-d) / (u-d) 
    
    # initialize stock values matrix as all zeros - we will only 
    # fill half the values
    stock = np.zeros((n+1,n+1))
    stock[0,0] = spot
    for i in range(1,n+1):
        stock[i,0] = u*stock[i-1,0]
        for j in range(1,i+1):
            stock[i,j] = d*stock[i-1,j-1]
    return stock

In [68]:
# visualize output
build_stock_tree(spot=1, tau=1.0, n_steps=10).round(1)

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1.1, 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1.2, 1. , 0.8, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1.3, 1.1, 0.9, 0.8, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1.5, 1.2, 1. , 0.8, 0.7, 0. , 0. , 0. , 0. , 0. , 0. ],
       [1.6, 1.3, 1.1, 0.9, 0.8, 0.6, 0. , 0. , 0. , 0. , 0. ],
       [1.8, 1.5, 1.2, 1. , 0.8, 0.7, 0.6, 0. , 0. , 0. , 0. ],
       [1.9, 1.6, 1.3, 1.1, 0.9, 0.8, 0.6, 0.5, 0. , 0. , 0. ],
       [2.1, 1.8, 1.5, 1.2, 1. , 0.8, 0.7, 0.6, 0.5, 0. , 0. ],
       [2.3, 1.9, 1.6, 1.3, 1.1, 0.9, 0.8, 0.6, 0.5, 0.4, 0. ],
       [2.6, 2.1, 1.8, 1.5, 1.2, 1. , 0.8, 0.7, 0.6, 0.5, 0.4]])

**Second step**: build pricer for European vanilla options

In [69]:
def european_pricer(option_type, stock, strike, tau, n_steps, risk_free_rate):
    
    assert option_type in ['call', 'put'], "Bad option type"
    
    # parameters
    dt = tau/n_steps
    u = np.exp(sigma*np.sqrt(dt))
    d = 1.0/u
    p = (np.exp(risk_free_rate*dt)-d) / (u-d) 

    # boundary conditions: options value at final node
    value = np.zeros((n+1, n+1))
    for j in range(n+1):
        if option_type == 'call':
            value[n, j] = max(stock[n_steps, j]- strike, 0)
        else: 
            value[n, j] = max(strike - stock[n_steps, j], 0)
            
    # now we just propagate back - since these are European options,
    # value at each node is just the propagated discounted expectation
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            value[i, j] = np.exp(-risk_free_rate*dt) * (p * value[i+1, j] + (1-p) * value[i+1, j+1])
    
    return value[0,0]

**Test**: to see if this works, we check whether the put-call parity $$c(t) - p(t) = S(t) - K e^{-r(T-t)}$$ holds

In [70]:
tau, n_steps, r, spot = 1.0, 10, 0.1, 100
strike = 100

stock = build_stock_tree(spot=spot, tau=tau, n_steps=n_steps)
call_price_eur = european_pricer('call', stock=stock, strike=strike, tau=tau, n_steps=n_steps, risk_free_rate=r)
print("Call price (European): ", round(call_price_eur, 4))
put_price_eur = european_pricer('put', stock=stock, strike=strike, tau=tau, n_steps=n_steps, risk_free_rate=r)
print("Put price (European): ", round(put_price_eur, 4))

Call price (European):  16.4405
Put price (European):  6.9242


In [71]:
# left-hand side
print(round(call_price_eur - put_price_eur,4))

9.5163


In [72]:
# right-hand side
print(round(spot - strike * np.exp(-r*tau),4))

9.5163


As we can see both match.

**Third step**: adjust code above for American options

In [56]:
def american_pricer(option_type, stock, strike, tau, n_steps, risk_free_rate):
    
    assert option_type in ['call', 'put'], "Bad option type"
    
    # parameters
    dt = tau/n_steps
    u = np.exp(sigma*np.sqrt(dt))
    d = 1.0/u
    p = (np.exp(risk_free_rate*dt)-d) / (u-d) 

    # boundary conditions: options value at final node
    value = np.zeros((n+1, n+1))
    for j in range(n+1):
        if option_type == 'call':
            value[n, j] = max(stock[n_steps, j]- strike, 0)
        else: 
            value[n, j] = max(strike - stock[n_steps, j], 0)
            
    # now we just propagate back and check on which is higher: implicit value 
    # or discounted propagated value
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            if option_type == 'call':
                value[i, j] = max(np.exp(-risk_free_rate*dt) * (p * value[i+1, j] + (1-p) * value[i+1, j+1]),
                                  max(stock[i,j] - strike, 0)
                                 )
            else:
                value[i, j] = max(np.exp(-risk_free_rate*dt) * (p * value[i+1, j] + (1-p) * value[i+1, j+1]),
                                  max(strike - stock[i,j], 0)
                                 )
    
    return value[0,0]

**Test**: for an American call without dividends, its value should be the same as the equivalent European one (i.e. optimal exercise is at maturity)

In [63]:
call_price_amer = american_pricer('call', stock=stock, strike=strike, tau=tau, n_steps=n_steps, risk_free_rate=r)
print("Call price (American): ", round(call_price_amer, 4))
print("Call price (European): ", round(call_price_eur, 4))

Call price (American):  16.4405
Call price (European):  16.4405


**Second test**: values for American puts should be greater or equal than the equivalent European option

In [64]:
put_price_amer = american_pricer('put', stock=stock, strike=strike, tau=tau, n_steps=n_steps, risk_free_rate=r)
print("Put price (American): ", round(put_price_amer, 4))
print("Put price (European): ", round(put_price_eur, 4))

Put price (American):  8.1963
Put price (European):  6.9242
